# 🗺️ SAT POI Viewer with WD + OSM
This notebook loads POIs (e.g. toilets, water points) from Wikidata with optional OSM matches and renders them on a Folium map with filtering options.

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import re

# Define the SPARQL endpoint and query
endpoint_url = "https://query.wikidata.org/sparql"
query = """
SELECT ?item ?itemLabel ?coord ?OSMnode ?OSMway ?OSMrel WHERE {
  ?item wdt:P6104 wd:Q134294510;  # part of Stockholm Archipelago Trail
        wdt:P625 ?coord.          # has coordinates
  OPTIONAL { ?item wdt:P10689 ?OSMway. }
  OPTIONAL { ?item wdt:P402 ?OSMrel. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'sv,en'. }
}
"""

# Run the SPARQL query
sparql = SPARQLWrapper(endpoint_url)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Parse the results
data = []

for res in results["results"]["bindings"]:
    qid = res["item"]["value"].split("/")[-1]
    label = res.get("itemLabel", {}).get("value", "")
    
    # Convert 'Point(lon lat)' to lat, lon
    coord = res["coord"]["value"]
    match = re.search(r"Point\(([-\d.]+) ([-\d.]+)\)", coord)
    if not match:
        continue  # Skip invalid coordinates
    lon, lat = float(match.group(1)), float(match.group(2))

    # Build OSM URL based on available ID
    osm_url = ""
    if "OSMrel" in res:
        osm_url = f"https://www.openstreetmap.org/relation/{res['OSMrel']['value']}"
    elif "OSMway" in res:
        osm_url = f"https://www.openstreetmap.org/way/{res['OSMway']['value']}"
    elif "OSMnode" in res:
        osm_url = f"https://www.openstreetmap.org/node/{res['OSMnode']['value']}"

    # Construct Wikidata URL
    wd_url = f"https://www.wikidata.org/wiki/{qid}"

    # Append to results
    data.append({
        "qid": qid,
        "label": label,
        "lat": lat,
        "lon": lon,
        "source": "Wikidata",
        "osm_url": osm_url,
        "wd_url": wd_url
    })

# Preview sample output
import pandas as pd
pd.DataFrame(data).head()


,qid,label,lat,lon,source,osm_url,wd_url
0,Q115303256,Ornö kyrkogård,59.052383,18.429910,Wikidata,https://www.openstreetmap.org/way/220390585,https://www.wikidata.org/wiki/Q115303256
1,Q115303257,Utö kyrkogård,58.957520,18.290876,Wikidata,https://www.openstreetmap.org/way/237190145,https://www.wikidata.org/wiki/Q115303257
2,Q115305305,Sandhamns kyrkogård,59.288432,18.906155,Wikidata,https://www.openstreetmap.org/way/701376877,https://www.wikidata.org/wiki/Q115305305
3,Q116730327,Båtluffarleden,59.469830,18.753580,Wikidata,https://www.openstreetmap.org/relation/8603180,https://www.wikidata.org/wiki/Q116730327
4,Q121352030,"Grillplats, Storsand Ålö",58.909430,18.219230,Wikidata,https://www.openstreetmap.org/node/1412263113,https://www.wikidata.org/wiki/Q121352030


In [2]:
import json

# Convert to GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": []
}

for item in data:
    feature = {
        "type": "Feature",
        "properties": {
            "qid": item["qid"],
            "label": item["label"],
            "source": item["source"],
            "osm_url": item["osm_url"],
            "wd_url": item["wd_url"]
        },
        "geometry": {
            "type": "Point",
            "coordinates": [item["lon"], item["lat"]]  # GeoJSON = [lon, lat]
        }
    }
    geojson["features"].append(feature)

# Save to file
with open("SAT_POIs.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("✅ GeoJSON file 'SAT_POIs.geojson' created.")


✅ GeoJSON file 'SAT_POIs.geojson' created.
